In [1162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from sklearn.linear_model import Lasso, Ridge
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import FeatureUnion, Pipeline
from datetime import date

In [1163]:
df=pd.read_csv('marketing_data.csv')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/14,0,189,...,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,"$57,091.00",0,0,6/15/14,0,464,...,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/14,0,134,...,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/14,0,10,...,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/14,0,6,...,2,7,1,0,0,0,0,1,0,SP


In [1164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   2240 non-null   int64 
 1   Year_Birth           2240 non-null   int64 
 2   Education            2240 non-null   object
 3   Marital_Status       2240 non-null   object
 4    Income              2216 non-null   object
 5   Kidhome              2240 non-null   int64 
 6   Teenhome             2240 non-null   int64 
 7   Dt_Customer          2240 non-null   object
 8   Recency              2240 non-null   int64 
 9   MntWines             2240 non-null   int64 
 10  MntFruits            2240 non-null   int64 
 11  MntMeatProducts      2240 non-null   int64 
 12  MntFishProducts      2240 non-null   int64 
 13  MntSweetProducts     2240 non-null   int64 
 14  MntGoldProds         2240 non-null   int64 
 15  NumDealsPurchases    2240 non-null   int64 
 16  NumWeb

In [1165]:
df.drop(['Complain'], axis=1, inplace=True)
df.rename({'Response':'AcceptedCmp6'}, axis=1, inplace=True)
df['TotalAmounts']=df['MntWines']+df['MntFruits']+df['MntMeatProducts']+df['MntFishProducts']+df['MntSweetProducts']+df['MntGoldProds']
df['TotalPurchases']=df['NumWebPurchases']+df['NumCatalogPurchases']+df['NumStorePurchases']
df['Children']=df['Kidhome'] + df['Teenhome']
df['Age']=2022-df['Year_Birth']
df.drop(['Kidhome', 'Teenhome', 'Year_Birth'], axis=1, inplace=True)

In [1166]:
#Clean income column (rename and change data type)
df.rename({' Income ':'Income'}, axis=1, inplace=True)
df['Income']=df['Income'].str.replace('$', '')
df['Income'] = df['Income'].str.replace(',', '')
df=df.astype({'Income':'float'})
df.Income.dtype

dtype('float64')

In [1167]:
#yearbirth to age


In [1168]:
#look for outliers/errors in Numerical/Categorical
categorical=df.select_dtypes(include='object')
numeric=df.select_dtypes(exclude='object')

In [ ]:
def report(a):
    fig, ax = plt.subplots(1,3)
    fig.set_size_inches(14,3)
    fig.suptitle(a, fontsize=16)
    sns.histplot(data=numeric, 
                 x=a, 
                 kde=True, 
                 ax=ax[0])
    sns.boxplot(data=numeric, 
                x=a, 
                ax=ax[1])
    sns.countplot(data=numeric, 
                 x=a,
                 palette='mako'
             )
    plt.show()
    print(numeric[a].describe())

for feature in numeric.columns:
    report(feature)

#histogram to check distribrutions, bar chart for clear outlier detection and countplot to ensure no floats in features 
#where only intergers would be expected. 

In [1170]:
numeric.query('Age > 100')

,ID,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,AcceptedCmp6,TotalAmounts,TotalPurchases,Children,Age
513,11004,60182.0,23,8,0,5,7,0,2,1,...,0,0,0,0,0,0,22,3,1,129
827,1150,83532.0,36,755,144,562,104,64,224,1,...,0,0,1,0,0,0,1853,14,0,123
2233,7829,36640.0,99,15,6,8,7,4,25,1,...,0,0,0,0,0,0,65,5,1,122


In [1171]:
#since number outliers are low and this notebook is designed solely for EDA/Data Cleaning, I'll just delete these rows. 
#I would impute instead of deletion if there were more values/I was preparing for proper modelling. (Using KNN or MICE)
numeric=numeric[numeric.Age<100]

In [1172]:
#same logic as above
numeric.query('Income > 500000')

,ID,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,AcceptedCmp6,TotalAmounts,TotalPurchases,Children,Age
527,9432,666666.0,23,9,14,18,8,1,12,4,...,0,0,0,0,0,0,62,7,1,45


In [1173]:
numeric.drop([527], inplace=True)

In [1174]:
#check for null values
df.isnull().sum() 

ID                      0
Education               0
Marital_Status          0
Income                 24
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
AcceptedCmp6            0
Country                 0
TotalAmounts            0
TotalPurchases          0
Children                0
Age                     0
dtype: int64

In [1175]:
#print null values for income to see anything discernable. 
numeric[numeric['Income'].isnull()]

,ID,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,AcceptedCmp6,TotalAmounts,TotalPurchases,Children,Age
134,8996,NaN,4,230,42,192,49,37,53,12,...,0,0,0,0,0,0,603,17,3,65
262,1994,NaN,11,5,5,6,0,2,1,1,...,0,0,0,0,0,0,19,3,1,39
394,3769,NaN,17,25,1,13,0,0,3,1,...,0,0,0,0,0,0,42,4,1,50
449,5255,NaN,19,5,1,3,3,263,362,0,...,0,0,0,0,0,0,637,27,1,36
525,8268,NaN,23,352,0,27,10,0,15,3,...,0,0,0,0,0,0,404,14,1,61
590,10629,NaN,25,25,3,43,17,4,17,3,...,0,0,0,0,0,0,109,6,1,49
899,10475,NaN,39,187,5,65,26,20,14,2,...,0,0,0,0,0,0,317,12,1,52
997,9235,NaN,45,7,0,8,2,0,1,1,...,0,0,0,0,0,0,18,3,2,65
1096,4345,NaN,49,5,1,9,2,0,4,1,...,0,0,0,0,0,0,21,3,2,58
1185,7187,NaN,52,375,42,48,94,66,96,7,...,0,0,0,0,0,0,721,18,2,53


In [1176]:
knn = KNNImputer()
numeric_processed=pd.DataFrame(knn.fit_transform(numeric))
numeric_processed.columns=numeric.columns
numeric_processed.isna().sum().sum()

0

In [1177]:
finaldf=pd.concat([numeric_processed, categorical],axis=1)

In [1178]:
for f in categorical.columns:
    print(categorical[f].value_counts())
    print('***********************************')

Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: Education, dtype: int64
***********************************
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
YOLO          2
Absurd        2
Name: Marital_Status, dtype: int64
***********************************
8/31/12    12
9/12/12    11
2/14/13    11
5/12/14    11
8/20/13    10
           ..
9/5/12      1
11/9/13     1
5/16/13     1
7/20/13     1
9/1/12      1
Name: Dt_Customer, Length: 663, dtype: int64
***********************************
SP     1095
SA      337
CA      268
AUS     160
IND     148
GER     120
US      109
ME        3
Name: Country, dtype: int64
***********************************


In [1179]:
#education has 2n cycle which I'm unsure what that means and documentation doesn't know either.
finaldf['Education']=finaldf['Education'].replace('2n Cycle', 'Graduation')
finaldf.Education.value_counts()

Graduation    1330
PhD            486
Master         370
Basic           54
Name: Education, dtype: int64

In [1180]:
def applyFunc(s):
    if s == 'Married':
        return 'No'
    elif s == 'Together':
        return 'No'
    else:
        return 'Yes'

finaldf['Single']=finaldf['Marital_Status'].apply(applyFunc)
finaldf.drop(['Marital_Status'], axis=1, inplace=True)
finaldf.Single.value_counts()

No     1444
Yes     796
Name: Single, dtype: int64

In [1191]:
finaldf['Country']=finaldf['Country'].replace({'SP':'Spain', 'SA': 'South Africa', 'CA':'Canada', 'AUS':'Australia', 'IND':'India', 'GER':'Germany', 'US':'USA'})
finaldf=finaldf[finaldf.Country != 'ME']
finaldf.Country.value_counts()

Spain           1095
South Africa     337
Canada           268
Australia        160
India            148
Germany          120
USA              109
Name: Country, dtype: int64

In [1183]:
finaldf.to_csv('Marketing.csv', index=False)

In [1184]:
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2237 entries, 0 to 2239
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2233 non-null   float64
 1   Income               2233 non-null   float64
 2   Recency              2233 non-null   float64
 3   MntWines             2233 non-null   float64
 4   MntFruits            2233 non-null   float64
 5   MntMeatProducts      2233 non-null   float64
 6   MntFishProducts      2233 non-null   float64
 7   MntSweetProducts     2233 non-null   float64
 8   MntGoldProds         2233 non-null   float64
 9   NumDealsPurchases    2233 non-null   float64
 10  NumWebPurchases      2233 non-null   float64
 11  NumCatalogPurchases  2233 non-null   float64
 12  NumStorePurchases    2233 non-null   float64
 13  NumWebVisitsMonth    2233 non-null   float64
 14  AcceptedCmp3         2233 non-null   float64
 15  AcceptedCmp4         2233 non-null   f

In [1190]:
finaldf

Dt_Customer     8/31/138/31/138/31/138/31/138/31/138/31/138/31...
TotalAmounts                                               7487.0
dtype: object